In [1]:
from estnltk import Text
import csv
from datetime import datetime
from tqdm import tqdm
import psycopg2
from connection_string_p12 import connection_string1, connection_string2
from collections import Counter
import pickle
import re

In [22]:
conn_egv = psycopg2.connect(connection_string1)
cur_egv = conn_egv.cursor()

In [23]:
cur_egv.execute("""SELECT diag_text_raw, diag_code from work.runfull201903041255_diagnosis 
                where diag_text_raw is not null and diag_code is not null;""")

In [24]:
texts = cur_egv.fetchall()

In [25]:
texts[0]

('II funktsionaalne klass', 'I20.8')

In [11]:
cd_texts = []
for t in texts:
    if t[1][0] == 'C' or (t[1][0] == 'D' and int(t[1][1]) < 5 ):
        cd_texts.append(t[0])

In [12]:
len(cd_texts)

27331

In [13]:
len(set(cd_texts))

7123

In [15]:
import sys
import os

In [16]:
sys.path.append(re.sub('cda_data_cleaning/fact_extraction/diag_text_parsing/CD_diag_explorations', '', os.getcwd()))

In [17]:
from cda_data_cleaning.fact_extraction.diag_text_parsing.development.taggers.cancer_stage_tagger import CancerStageTagger

INFO:base_parser.py:30: Loaded []


In [18]:
tagger = CancerStageTagger()

In [19]:
tagged_texts = []
untagged_texts = []
for t in cd_texts:
    text = Text(t)
    tagger.tag(text)
    if len(text.stages) > 0:
        tagged_texts.append(text)
    else:
        untagged_texts.append(text)

In [20]:
len(tagged_texts)

2268

In [26]:
for text in tagged_texts[:500]:
    for stage in text.stages:
        if stage.start > 50:
            start = stage.start
        else:
            start = 0
        if stage.end < len(text.text)-50:
            end = stage.end + 50
        else:
            end = len(text.text)
        context = text.text[start:end]
        print(stage.text)
        print(context)
        print()

IV st
Carcinoma ventriculi recidiva rp T0N0M1 IV st

IV st
Carcinoma recti cT4NxM1G2 IV st

IV st
Carcinoma recti cT4NxM1G2 IV st

IV st
Carcinoma recti cT4NxM1G2 IV st

st IV
st IV

st.IV
Ca vesicae urinariae pT4N2M0 G2 st.IV

st. IV
Carcinoma rectosigmoidale st. IV

st.Ib
Carcinoma colli uteri st.Ib1  pT1b1 pNo Mo G2

st IV 
BronchoCa! pulm dex lobi inf cT4cN2cM1b st IV (HEP) 

st.I 
Carcinoma mammae dex st.I pT1 pNo Mo G2

st.IIIc
Carcinoma ovarii st.IIIc  pT2c pN1 Mo G3

IVBst
Ca coli sigmoidei pT3NxM1b IVBst.

st  IIIB
BronchoCa! periph lobi sup pulmonis sin  st  IIIB   cT1b  cN3   cMx

st IV
Ca laryngis supraglottidis ycT0ycN0pM1(PUL) st IV

IVst
Carcinoma rectosigmoidale IVst.

IVst
IVst.

IVst
Carcinoma rectosigmoidale IVst.

st IV
Carcinoma prostata pT4 N0 M1 st IV

st IV
Carcinoma ovariorum generalisata  st IV

IVst
Carcinoma coli ascendentis pT3pN1M1(LYM;PUL) IVst.

st.0 
Carcinoma in situ mammae dex st.0 pTis pNo Mo G3

st.Ia
Sarcoma corporis uteri st.Ia  pT1a pNo Mo G1

st